In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split, KFold, StratifiedGroupKFold
from torch.nn.utils.rnn import pad_sequence
import cv2
import h5py
import io
import pandas.api.types
import random
import sklearn.metrics
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import pandas.api.types
import sklearn.metrics
from math import sin,cos,pi
from sklearn.metrics import roc_auc_score, auc, roc_curve
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from transformers import get_cosine_schedule_with_warmup
from catboost import CatBoostClassifier,Pool,cv
from copy import deepcopy
import wandb
import timm 
pl.seed_everything(56)

Seed set to 56


56

In [2]:
class CFG:
    class data:
        train_data= './isic-2024-challenge/train-metadata.csv'
        train_hdf5='./isic-2024-challenge/train-image.hdf5'
        num_workers = 8
        img_size = 384
        nfolds = 5
        batch_size = 32
        seed = 56
    class model:
        model = 'efficientnetv2_rw_m.agc_in1k'
        pretrained = True
        optim = torch.optim.AdamW
        global_pool = 'avg' # 'avg', 'max', 'avgmax', 'catavg'
        drop_path_rate = 0.2
        cls_drop = 0.2
        num_chanels = 3
        num_labels = 2
        hidden_size = 2152
        scheduler = 'cosine'
        head_drop = 0.05
        max_epoches = 10
        lr = 1e-4
        num_cycles = 0.5
        warmup_ratio = 0.0
        lr_head = 1e-4
        eps = 1e-12
        weight_decay = 0.0
        weight_decay_head = 0.0
        betas = (0.9, 0.999)
    seed = 56
    fold_number = 4
    
class Transforms:
    transforms_train = A.Compose([
            A.Resize(CFG.data.img_size,CFG.data.img_size),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0),
            ToTensorV2()
        ])
    transforms_val = A.Compose([
            A.Resize(CFG.data.img_size,CFG.data.img_size),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0),
            ToTensorV2()
        ])
    transforms_test = A.Compose([
            A.Resize(CFG.data.img_size,CFG.data.img_size),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0),
            ToTensorV2()
        ])

def set_wandb_cfg():
    config = {}
    for k,v in CFG.model.__dict__.items():
        if '__' not in k:
            config[k] = v
    for k,v in CFG.data.__dict__.items():
        if '__' not in k:
            config[k] = v
    config['fold_number'] = CFG.fold_number
    return config

In [3]:
def make_df(path):
    data = pd.read_csv(path)
    return data[['isic_id','target','patient_id']]

In [4]:
class PLDataset(Dataset):
    def __init__(self, df, transforms, reader):
        super().__init__()
        self.cfg = CFG.data
        self.data = df
        self.transforms = transforms
        self.hdf5_filles = reader
        
    def __getitem__(self, index):
        row = self.data.iloc[index]
        image = np.array(Image.open(io.BytesIO(self.hdf5_filles[row['isic_id']][()])))
        image = self.transforms(image=image)['image']
            
        return {
            'image': image.squeeze(0),
            'labels': row['target']
        }
    
    def __len__(self):
        return len(self.data)

In [5]:
class PLDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.data
        self.is_setup = False
        self.is_prepared = False
        
    def prepare_data(self):
        if self.is_prepared: return None
        self.df = make_df(self.cfg.train_data)
        self.transforms = Transforms
        self.reader = h5py.File(self.cfg.train_hdf5,'r')
        self.is_prepared = True
        
    def setup(self, stage: str):
        if self.is_setup: return None
        kf = StratifiedGroupKFold(n_splits=self.cfg.nfolds, shuffle=True, random_state=self.cfg.seed)
        splits = [(x,y) for x,y in  kf.split(self.df,self.df['target'],self.df['patient_id'])][CFG.fold_number]
        self.train_df, self.val_df = self.df.iloc[splits[0]], self.df.iloc[splits[1]]
        self.train_dataset = PLDataset(self.train_df,self.transforms.transforms_train,self.reader)
        self.val_dataset = PLDataset(self.val_df,self.transforms.transforms_val,self.reader)
        #self.predict_dataset = PLDataset(self.test_df,self.transforms.transforms_test,self.reader)
        self.is_setup = True
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                         batch_size=self.cfg.batch_size,
                         num_workers=self.cfg.num_workers,
                         pin_memory=True,
                         shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          pin_memory=True,
                          shuffle=False)
    
    def predict_dataloader(self):
        return DataLoader(self.predict_dataset,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          pin_memory=True,
                          shuffle=False)

In [6]:
def p_auc_tpr(v_gt, v_pred, min_tpr=0.80, sample_weight=None):
    """Computes the area under the AUC above a minumum TPR.

    Args:
        v_gt: ground truth vector (1s and 0s)
        v_p: predictions vector of scores ranging [0, 1]
        min_tpr: minimum true positive threshold (sensitivity)

    Returns:
        Float value range [0, 1]
    """
    if len(np.unique(v_gt)) != 2:
        raise ValueError(
            "Only one class present in y_true. ROC AUC score "
            "is not defined in that case."
        )
    
    # redefine the target. set 0s to 1s and 1s to 0s
    v_gt = abs(np.asarray(v_gt)-1)
    v_pred = abs(np.asarray(v_pred)-1)
    max_fpr = abs(1-min_tpr)
    
    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=sample_weight)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)

    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)
    return(partial_auc)

In [7]:
class AverageMeter():
    def __init__(self):
        self.preds = []
        self.labels = []
        self.preds_round = []
        self.history = []
    
    def update(self,y_t,y_p,y_pr):
        self.labels += y_t
        self.preds_round += y_pr
        self.preds += y_p
        
    def clean(self):
        self.preds = []
        self.labels = []
        self.preds_round = []

    def calc_metrics(self):
        metrics = {}
        try:
            metrics['prauc'] = p_auc_tpr(self.labels,np.stack(self.preds)[:,1])
        except:
            metrics['prauc'] = -100
        self.history.append(metrics)
        return metrics

In [8]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.cfg = CFG.model
        self.encoder = timm.create_model(
                self.cfg.model,
                pretrained=self.cfg.pretrained,
                in_chans=self.cfg.num_chanels,
                num_classes=-1,
                drop_path_rate=self.cfg.drop_path_rate,
                global_pool=self.cfg.global_pool
        )
        self.cls_drop = nn.Dropout(self.cfg.cls_drop)
        self.fc = nn.Linear(self.cfg.hidden_size, self.cfg.num_labels)
        self._init_weights(self.fc)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def forward(self, image, return_features=False):
        features = self.encoder(image)
        if return_features:
            return features
        logits = self.fc(self.cls_drop(features))
        return logits

In [9]:
class PLModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.model
        self.model = Model()
        self.avg_meter = AverageMeter()
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, batch):
        output = self.model(batch['image'])
        return output

    def training_step(self, batch, i):
        logits = self(batch)
        loss = self.criterion(logits,batch['labels'])
        self.log('train_loss', loss.item())
        return loss
            
    def validation_step(self, batch, i):
        logits = self(batch)
        loss = self.criterion(logits,batch['labels'])
        self.log('val_loss',loss.item())
        preds = logits.argmax(dim=-1).tolist()
        self.avg_meter.update(batch['labels'].tolist(),logits.tolist(),preds)
    
    def predict_step(self, batch, i):
        logits = self(batch)
        return logits.softmax(dim=-1).tolist()
                
    def on_validation_epoch_end(self):
        metrics = self.avg_meter.calc_metrics()
        self.log_dict(metrics)
        self.avg_meter.clean()
            
    def configure_optimizers(self):        
        optimizer_parameters = [
            {'params': self.model.encoder.parameters(),
             'lr': self.cfg.lr, 'weight_decay': self.cfg.weight_decay},
            {'params': self.model.fc.parameters(),
             'lr': self.cfg.lr_head, 'weight_decay': self.cfg.weight_decay_head}
        ]
        
        optim = self.cfg.optim(
            optimizer_parameters,
            lr=self.cfg.lr,
            betas=self.cfg.betas,
            weight_decay=self.cfg.weight_decay,
            eps=self.cfg.eps
        )
        
        if self.cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warmup_ratio,
                                                        num_cycles=self.cfg.num_cycles)
        elif self.cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warmup_ratio)
        else:
            return optim
        
        scheduler = {'scheduler': scheduler,'interval': 'step', 'frequency': 1}

        return [optim], [scheduler]

In [ ]:
dm = PLDataModule()
dm.prepare_data()
dm.setup(0)

In [ ]:
CFG.model.num_training_steps = len(dm.train_dataloader()) * CFG.model.max_epoches

In [27]:
model = PLModule()

INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/efficientnetv2_rw_m.agc_in1k)
INFO:timm.models._hub:[timm/efficientnetv2_rw_m.agc_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


In [16]:
wandb.login(key="31520b01739d418e5d77a11fd8a79a70b189b8bc")
os.environ['WANDB_API_KEY'] = "31520b01739d418e5d77a11fd8a79a70b189b8bc"
wandb.init(project='Kaggle_ISIC',name='effnet_medium',config=set_wandb_cfg())

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: andrewkhl (andlh). Use `wandb login --relogin` to force relogin


In [28]:
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='step')
checkpoint_cb = pl.callbacks.ModelCheckpoint(
    dirpath='./outputs/',
    filename='model_{epoch:02d}-{prauc:.4f}',
    monitor='prauc',
    mode='max',
    save_last=True
)

trainer = pl.Trainer(
    accelerator="gpu",
    precision=32,
    callbacks = [lr_monitor,checkpoint_cb],
    logger = pl.loggers.WandbLogger(save_code=True),
    log_every_n_steps=1,
    min_epochs=1,
    devices=1,
    check_val_every_n_epoch=1,
    max_epochs=CFG.model.max_epoches
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [29]:
CFG.fold_number = 0

dm = PLDataModule()
dm.prepare_data()
dm.setup(0)

/tmp/ipykernel_45/2786575331.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [30]:
preds_fold_0 = trainer.predict(model,dm.val_dataloader(),ckpt_path="outputs_0/model_epoch=01-prauc=0.1420.ckpt")

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
Restoring states from the checkpoint path at outputs_0/model_epoch=01-prauc=0.1420.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at outputs_0/model_epoch=01-prauc=0.1420.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

In [38]:
CFG.fold_number = 1

dm = PLDataModule()
dm.prepare_data()
dm.setup(0)

/tmp/ipykernel_45/2786575331.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [39]:
preds_fold_1 = trainer.predict(model,dm.val_dataloader(),ckpt_path="outputs_1/model_epoch=00-prauc=0.1286.ckpt")

Restoring states from the checkpoint path at outputs_1/model_epoch=00-prauc=0.1286.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at outputs_1/model_epoch=00-prauc=0.1286.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

In [35]:
CFG.fold_number = 2

dm = PLDataModule()
dm.prepare_data()
dm.setup(0)

/tmp/ipykernel_45/2786575331.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [36]:
preds_fold_2 = trainer.predict(model,dm.val_dataloader(),ckpt_path="outputs_2/model_epoch=00-prauc=0.1182.ckpt")

Restoring states from the checkpoint path at outputs_2/model_epoch=00-prauc=0.1182.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at outputs_2/model_epoch=00-prauc=0.1182.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

In [40]:
CFG.fold_number = 3

dm = PLDataModule()
dm.prepare_data()
dm.setup(0)

/tmp/ipykernel_45/2786575331.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [41]:
preds_fold_3 = trainer.predict(model,dm.val_dataloader(),ckpt_path="outputs/model_epoch=01-prauc=0.1401.ckpt")

Restoring states from the checkpoint path at outputs/model_epoch=01-prauc=0.1401.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at outputs/model_epoch=01-prauc=0.1401.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

In [42]:
CFG.fold_number = 4

dm = PLDataModule()
dm.prepare_data()
dm.setup(0)

/tmp/ipykernel_45/2786575331.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [43]:
preds_fold_4 = trainer.predict(model,dm.val_dataloader(),ckpt_path="outputs/model_epoch=01-prauc=0.1257.ckpt")

Restoring states from the checkpoint path at outputs/model_epoch=01-prauc=0.1257.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at outputs/model_epoch=01-prauc=0.1257.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

In [52]:
df = dm.df.copy()

kf = StratifiedGroupKFold(n_splits=dm.cfg.nfolds, shuffle=True, random_state=dm.cfg.seed)
splits = [(x,y) for x,y in  kf.split(df,df['target'],df['patient_id'])]

In [63]:
df['fold'] = -1
df.loc[splits[0][1],'fold'] = 0
df.loc[splits[1][1],'fold'] = 1
df.loc[splits[2][1],'fold'] = 2
df.loc[splits[3][1],'fold'] = 3
df.loc[splits[4][1],'fold'] = 4

In [70]:
np.concatenate(preds_fold_4)[:,1]

array([2.31417289e-05, 1.26163577e-04, 2.19719863e-04, ...,
       6.85076157e-05, 1.01727387e-03, 3.48700996e-04])

In [71]:
df['predict'] = -1
df.loc[splits[0][1],'predict'] = np.concatenate(preds_fold_0)[:,1]
df.loc[splits[1][1],'predict'] = np.concatenate(preds_fold_1)[:,1]
df.loc[splits[2][1],'predict'] = np.concatenate(preds_fold_2)[:,1]
df.loc[splits[3][1],'predict'] = np.concatenate(preds_fold_3)[:,1]
df.loc[splits[4][1],'predict'] = np.concatenate(preds_fold_4)[:,1]

/tmp/ipykernel_45/1382868842.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[8.61431588e-04 1.04059407e-04 1.90227700e-04 ... 4.27426246e-04
 7.16760987e-05 7.33906927e-05]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[splits[0][1],'predict'] = np.concatenate(preds_fold_0)[:,1]


In [79]:
from sklearn.metrics import roc_auc_score, auc, roc_curve

In [82]:
def p_auc_tpr(v_gt, v_pred, min_tpr=0.80, sample_weight=None):
    """Computes the area under the AUC above a minumum TPR.

    Args:
        v_gt: ground truth vector (1s and 0s)
        v_p: predictions vector of scores ranging [0, 1]
        min_tpr: minimum true positive threshold (sensitivity)

    Returns:
        Float value range [0, 1]
    """
    if len(np.unique(v_gt)) != 2:
        raise ValueError(
            "Only one class present in y_true. ROC AUC score "
            "is not defined in that case."
        )
    
    # redefine the target. set 0s to 1s and 1s to 0s
    v_gt = abs(np.asarray(v_gt)-1)
    v_pred = abs(np.asarray(v_pred)-1)
    max_fpr = abs(1-min_tpr)
    
    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=sample_weight)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)

    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)
    return(partial_auc)

In [83]:
p_auc_tpr(df['target'],df['predict'])

0.1230816492067425

In [76]:
df[['target','predict']]

,target,predict
0,0,0.000740
1,0,0.003848
2,0,0.000113
3,0,0.000126
4,0,0.000861
...,...,...
401054,0,0.000210
401055,0,0.000138
401056,0,0.000072
401057,0,0.000073


In [85]:
df.to_csv('simple_effnet_preds.csv',index=False)